<a href="https://colab.research.google.com/github/PrabhuRajendhran/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example_updated_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
%%capture
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

# from huggingface_hub import login
# login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset-1"
finetune_tags = ["smol-course", "module_1"]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

In [3]:
device

'cpu'

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [4]:
# Let's test the base model before training
prompt = "Write a python fuction to print 'hello world'"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a python fuction to print 'hello world'
Write a python fuction to print 'hello world'
Write a python fuction to print 'hello world'
Write a python fuction to print 'hello world'
Write a python fuction to print 'hello world'
Write a python fuction to print 'hello world'
Write a python fuction to print 'hello world'
Write a python fuction to print 'hello world'
Write a python fuction to print 'hello world'
Write a python fu


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [5]:
from datasets import load_dataset
dataset = load_dataset("bigcode/the-stack-smol", data_dir="data/python")

README.md:   0%|          | 0.00/3.30k [00:00<?, ?B/s]

data.json:   0%|          | 0.00/87.0M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
# def extract_docstring(content):
#   lines = content.split('\n')
#   docstring = ""
#   in_docstring = False
#   for line in lines:
#       if '"""' in line or "'''" in line:
#           if not in_docstring:
#               in_docstring = True
#               docstring += line.strip() + " "
#           else:
#               docstring += line.strip()
#               break
#       elif in_docstring:
#           docstring += line.strip() + " "
#   return docstring.strip()

import re

def extract_docstring(content):
    match = re.search(r'"""(.*?)"""', content, re.DOTALL)  # Using non-greedy match
    if match:
        return match.group(1).strip()
    else:
        return "No docstring found."

def preprocess_function(examples):
    prompts = [extract_docstring(content) for content in examples['content']]
    return {"question": prompts, "answer": examples['content']}

# Apply preprocessing
ds = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [22]:
ds['train']['content'][1]

'import scipy.sparse as sp\nimport numpy as np\nimport torch\nimport time\nimport os\nfrom configparser import ConfigParser\n\nimport sys\nsys.path.append(\'/home/shiyan/project/gcn_for_prediction_of_protein_interactions/\')\n\nfrom src.util.load_data import load_data, sparse_to_tuple, mask_test_edges, preprocess_graph\nfrom src.util.loss import arga_loss_function, varga_loss_function\nfrom src.util.metrics import get_roc_score\nfrom src.util import define_optimizer\nfrom src.graph_nheads_att_gan.model import NHGATModelGAN\n\nDEVICE = torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\')\n\nclass Train():\n    def __init__(self):\n        pass\n\n    def train_model(self, config_path):\n        if os.path.exists(config_path) and (os.path.split(config_path)[1].split(\'.\')[0] == \'config\') and (\n                os.path.splitext(config_path)[1].split(\'.\')[1] == \'cfg\'):\n            # load config file\n            config = ConfigParser()\n            config.read(config_pa

In [7]:
def convert_to_chat_format(sample):
    question = sample['question']
    answer = sample['answer']

    messages = [
        {'content': question, 'role': 'user'},
        {'content': answer, 'role': 'assistant'}
    ]

    sample['messages'] = messages

    return sample

def process_dataset(sample):
    # TODO: 🐕 Convert the sample into a chat format

    # 1. create a message format with the role and content

    sample = convert_to_chat_format(sample)

    # 2. apply the chat template to the samples using the tokenizer's method

    chat_format = tokenizer.apply_chat_template(sample['messages'], add_generation_prompt=True)
    sample['chat_format'] = chat_format

    return sample


ds = ds.map(process_dataset)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (8732 > 8192). Running this sequence through the model will result in indexing errors


## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-05f22f6eec0e>:18: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying chat template to train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2260 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/119 [00:00<?, ? examples/s]

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: prabhurajendren8 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
50,1.065700,1.158981
100,1.111600,1.124065
150,1.062400,1.095485
200,1.048200,1.079699
250,1.041200,1.070457
300,1.029200,1.061473
350,1.003400,1.054751
400,1.006500,1.050794
450,1.021100,1.042637
500,1.076200,1.033726


In [ ]:
trainer.push_to_hub(tags=finetune_tags)

events.out.tfevents.1739600650.f8c77d88fd0c.824.0:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PrabhuRajendren/SmolLM2-FT-MyDataset/commit/475a5f9c5dc219c29ca04be260716bcb77316991', commit_message='End of training', commit_description='', oid='475a5f9c5dc219c29ca04be260716bcb77316991', pr_url=None, repo_url=RepoUrl('https://huggingface.co/PrabhuRajendren/SmolLM2-FT-MyDataset', endpoint='https://huggingface.co', repo_type='model', repo_id='PrabhuRajendren/SmolLM2-FT-MyDataset'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Step 1: Load the Fine-Tuned Model
model_name = "/content/SmolLM2-FT-MyDataset-1" #"./sft_output"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model = model.to(device)
#tokenizer = tokenizer.to(device)

In [ ]:
# Let's test the base model after training
prompt = "Write a python fuction to print 'hello world'"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
Write a haiku about programming
assistant
Hello! How can I help you today? I'm going to write a haiku about programming. What's programming? It's a type of art where you create a sequence of steps to solve a problem. Think of it like a recipe. You follow the steps, and then you see the result. Can you think of any programming languages? Python, Java, or JavaScript are all popular ones. What's the most popular programming language right now? Python is very popular,


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.